# Self-Attention

In [1]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_packed_sequence #PyTorch中处理变长序列的重要函数

In [2]:
class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        # 初始化
        self.data_dir = data_dir
        self.segment_len = segment_len

        # 加载json格式的映射文件（说话人到ID的映射关系）
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # 从metadata.json 文件中加载训练数据的说话人的元数据
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        self.speaker_num = len(metadata.keys()) # 说话人的总数
        self.data = [] # 初始化数据容器
        # 遍历每个说话人及其所有utterances（语言片段）
        for speaker in metadata.keys(): 
            for utterances in metadata[speaker]:
                # 存储：[特征路径, 说话人ID]
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        
        mel = torch.load(os.path.join(self.data_dir, feat_path)) # 加载预处理好的梅尔频谱数据

        # 检查梅尔频谱长度是否大于目标分段长度
        if len(mel) > self.segment_len:
            start = random.randint(0, len(mel) - self.segment_len) # 随机选取起始点
            mel = torch.FloatTensor(mel[start:start+self.segment_len]) # 截取segment_len长度的片段
        else:
            mel = torch.FloatTensor(mel) #若长度不足,直接转换为张量
            # 注:长度不足segment_len时直接转换,可能导致模型报错,需要填充短序列至segment_len

        speaker = torch.FloatTensor([speaker]).long() # 将说话人ID转化为PyTorch的长整型张量

        return mel, speaker
    
    def get_speaker_number(self):
        return self.speaker_num



In [3]:
import torch
from torch.utils.data import DataLoader, random_split # random_split随机分割
from torch.nn.utils.rnn import pad_sequence # pad_sequence序列填充

In [4]:
def collate_batch(batch):
    """处理一个批次的数据"""
    mel, speaker = zip(*batch) # 解压批次数据

    mel = pad_sequence(mel, batch_first=True, padding_value=-20) # 填充梅尔频谱(用-20填充短序列)

    return mel, torch.FloatTensor(speaker).long() # 处理说话人标签

In [5]:
def get_dataloader(data_dir, batch_size, n_workers):
    dataset = myDataset(data_dir)
    speaker_num = dataset.get_speaker_number()

    trainlen = int(0.9 * len(dataset)) # 样本数不为整数向下取整, 剩余部分分配给验证集
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size, # 每批次样本数
        shuffle=True,  # 是否打乱数据顺序
        drop_last=True,
        num_workers=n_workers, # 并行加载数据传输
        pin_memory=True, # 是否锁页内存
        collate_fn=collate_batch, # 自定义批处理函数
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers, 
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Classifier(nn.Module):
    
    def __init__(self, d_model=80, n_spk=600, dropout=0.1):
        """
            说话人分类器模型
            Args:
                d_model:模型特征维度
                n_spk:说话人数量
                dropout: Dropout率
        """
        # 前置网咯
        super().__init__()
        
        # 输入: (batch, seq_len, 40) -> 输出: (batch, seq_len, d_model)
        self.prenet = nn.Linear(40, d_model) # 将梅尔频谱特征投影到模型维度

        # Transformer编码层
        # nhead: 多头注意力头数 dim_feadforward:前馈网络隐藏层维度
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, dim_feedforward=256, nhead=2
        )

        # 分类预测层
        # 结构：线性层 -> ReLU -> 线性层
        # 输入: (batch, d_model) -> 输出: (batch, n_spk)
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, n_spk),
        )

    def forward(self, mels):

        # 1. 特征投影
        # (batch, seq_len, 40) -> (batch, seq_len, d_model)
        out = self.prenet(mels)
        
        # 2. 调整维度顺序以适应Transformer输入要求
        # (batch, seq_len, d_model) -> (seq_len, batch, d_model)
        out = out.permute(1, 0, 2)
        
        # 3. Transformer特征编码
        # 输入输出形状保持: (seq_len, batch, d_model)
        out = self.encoder_layer(out)
        
        # 4. 恢复原始维度顺序
        # (seq_len, batch, d_model) -> (batch, seq_len, d_model)
        out = out.transpose(0, 1)
        
        # 5. 时序平均池化（统计聚合）
        # (batch, seq_len, d_model) -> (batch, d_model)
        stats = out.mean(dim=1)
        
        # 6. 分类预测
        # (batch, d_model) -> (batch, n_spk)
        out = self.pred_layer(stats)
        return out

In [8]:
print(Classifier())

Classifier(
  (prenet): Linear(in_features=40, out_features=80, bias=True)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=80, out_features=80, bias=True)
    )
    (linear1): Linear(in_features=80, out_features=256, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=256, out_features=80, bias=True)
    (norm1): LayerNorm((80,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((80,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (pred_layer): Sequential(
    (0): Linear(in_features=80, out_features=80, bias=True)
    (1): ReLU()
    (2): Linear(in_features=80, out_features=600, bias=True)
  )
)


In [9]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR

In [10]:
# 带预热的余弦学习率调度器
def get_cosine_schedule_with_warmup(
        optimizer: Optimizer, # 要优化的目标
        num_warmup_steps: int, # 预热步数
        num_training_steps: int, # 总训练步数
        num_cycles: float = 0.5, # 余弦周期数
        last_epoch: int = -1, # 恢复训练时的epoch标记
):

    def  lr_lambda(current_step):
        # 1. 预热阶段:线性增长
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        
        # 2. 计算退火进度
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        # 3. 余弦退火计算
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))
    return LambdaLR(optimizer, lr_lambda, last_epoch)

In [11]:
import torch
def model_fn(batch, model, criterion, device):
    """执行单批次的前向传播和指标计算
    Args:
        batch: 包含(mels, labels)的元组
        model: 神经网络模型
        criterion: 损失函数
        device: 计算设备(cpu/cuda)
    Returns:
        loss: 当前批次的损失值
        accuracy: 当前批次的准确率
    """
    # 1. 数据准备
    mels, labels = batch
    mels = mels.to(device)      # 梅尔频谱特征 -> GPU
    labels = labels.to(device)  # 说话人标签 -> GPU

    # 2. 前向传播
    outs = model(mels)          # 模型预测输出

    # 3. 损失计算
    loss = criterion(outs, labels)

    # 4. 指标计算
    with torch.no_grad():       # 禁用梯度以节省内存
        preds = outs.argmax(dim=1)           # 获取预测类别
        accuracy = (preds == labels).float().mean()  # 计算准确率

    return loss, accuracy

In [12]:
from tqdm import tqdm
import torch

def valid(dataloader, model, criterion, device):
    # 1. 设置模型为评估模式
    model.eval()
    
    # 2. 初始化累计指标
    running_loss = 0.0
    running_accuracy = 0.0
    
    # 3. 创建进度条
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

    # 4. 批次循环
    for i, batch in enumerate(dataloader):
        # 禁用自动求导
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
        
        # 累计指标
        running_loss += loss.item()
        running_accuracy += accuracy.item()

        # 更新进度条
        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    # 5. 清理和返回
    pbar.close()
    model.train()  # 恢复训练模式
    return running_accuracy / len(dataloader)

In [13]:
from tqdm import tqdm # 导入tqdm库，用于显示训练进度条

import torch # 导入PyTorch库
import torch.nn as nn # 导入神经网络模块
from torch.optim import AdamW # 导入AdamW优化器
from torch.utils.data import DataLoader, random_split # 导入数据加载器和随机分割数据集的工具

# 假设 Classifier, get_cosine_schedule_with_warmup, model_fn, valid, get_dataloader 已在其他地方定义
# from your_model_file import Classifier # 假设Classifier模型定义在your_model_file.py中
# from your_scheduler_file import get_cosine_schedule_with_warmup # 假设调度器定义在your_scheduler_file.py中
# from your_utils_file import model_fn, valid, get_dataloader # 假设其他辅助函数定义在your_utils_file.py中


def parse_args():
  """
  解析训练参数的函数。
  通常用于从命令行或配置文件读取参数，这里直接定义了一个字典作为配置。
  """
  config = {
    "data_dir": "../../Data/ml2021spring-hw4/Dataset", # 数据集所在的目录
    "save_path": "model.ckpt", # 模型保存的路径
    "batch_size": 32, # 批次大小
    "n_workers": 8, # 数据加载器的工作进程数
    "valid_steps": 2000, # 每隔多少训练步进行一次验证
    "warmup_steps": 1000, # 学习率预热的步数
    "save_steps": 10000, # 每隔多少训练步保存一次最佳模型
    "total_steps": 70000, # 总的训练步数
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """
  主训练函数。
  负责整个模型的训练流程，包括数据加载、模型初始化、训练循环、验证和模型保存。
  """
  # 1. 设备设置
  # 检查是否有可用的CUDA（GPU），如果有则使用GPU，否则使用CPU
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!") # 打印当前使用的设备

  # 2. 数据加载
  # 获取训练数据加载器、验证数据加载器和说话人数量
  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  # 创建训练数据加载器的迭代器，以便在训练循环中通过next()获取批次数据
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True) # 打印数据加载完成信息

  # 3. 模型、损失函数、优化器和调度器初始化
  # 初始化分类器模型，并将其移动到指定设备
  model = Classifier(n_spk=speaker_num).to(device)
  # 定义交叉熵损失函数，适用于多分类任务
  criterion = nn.CrossEntropyLoss()
  # 定义AdamW优化器，用于更新模型参数，学习率初始设置为1e-3
  optimizer = AdamW(model.parameters(), lr=1e-3)
  # 创建带预热的余弦学习率调度器
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True) # 打印模型创建完成信息

  # 4. 训练状态初始化
  best_accuracy = -1.0 # 记录迄今为止最佳的验证准确率
  best_state_dict = None # 存储最佳模型的参数字典

  # 5. 训练进度条初始化
  # 创建一个训练进度条，总步数为valid_steps（每valid_steps更新一次进度条）
  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  # 6. 主训练循环
  # 循环执行total_steps次训练步
  for step in range(total_steps):
    # 6.1 获取训练数据批次
    try:
      batch = next(train_iterator) # 尝试从迭代器中获取下一个批次
    except StopIteration:
      # 如果迭代器耗尽（一个epoch结束），则重新创建迭代器，开始新的epoch
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    # 6.2 前向传播、损失和准确率计算
    # 调用model_fn处理当前批次数据，返回损失和准确率
    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item() # 获取当前批次的损失值（Python浮点数）
    batch_accuracy = accuracy.item() # 获取当前批次的准确率（Python浮点数）

    # 6.3 模型参数更新
    loss.backward() # 反向传播，计算梯度
    optimizer.step() # 根据梯度更新模型参数
    scheduler.step() # 更新学习率（根据调度器策略）
    optimizer.zero_grad() # 清零梯度，为下一个批次的计算做准备

    # 6.4 训练日志记录和进度条更新
    pbar.update() # 更新进度条的当前步数
    pbar.set_postfix( # 设置进度条右侧的额外信息
      loss=f"{batch_loss:.2f}", # 显示当前批次的损失
      accuracy=f"{batch_accuracy:.2f}", # 显示当前批次的准确率
      step=step + 1, # 显示当前训练步数
    )

    # 6.5 执行验证
    if (step + 1) % valid_steps == 0: # 每隔valid_steps步进行一次验证
      pbar.close() # 关闭当前的训练进度条

      # 调用valid函数在验证集上评估模型，并获取验证准确率
      valid_accuracy = valid(valid_loader, model, criterion, device)

      # 6.6 保存最佳模型
      if valid_accuracy > best_accuracy: # 如果当前验证准确率优于历史最佳
        best_accuracy = valid_accuracy # 更新最佳准确率
        best_state_dict = model.state_dict() # 保存当前模型的参数字典

      # 重新初始化训练进度条，开始下一个训练阶段的计数
      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # 6.7 定期保存最佳模型到磁盘
    # 每隔save_steps步，并且已经有最佳模型时，保存模型
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path) # 将最佳模型参数保存到指定路径
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})") # 在进度条下方打印保存信息

  # 7. 训练结束清理
  pbar.close() # 训练循环结束后，关闭最终的进度条


if __name__ == "__main__":
  # 当脚本作为主程序运行时，执行main函数
  # parse_args() 获取配置字典，**将其解包为关键字参数传递给main函数
  main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [00:09<00:00, 206.83 step/s, accuracy=0.22, loss=3.55, step=2000]
Valid: 100% 6944/6944 [00:00<00:00, 8304.18 uttr/s, accuracy=0.21, loss=3.85]
Train: 100% 2000/2000 [00:04<00:00, 439.93 step/s, accuracy=0.31, loss=3.31, step=4000]
Valid: 100% 6944/6944 [00:00<00:00, 15766.01 uttr/s, accuracy=0.31, loss=3.21]
Train: 100% 2000/2000 [00:04<00:00, 438.50 step/s, accuracy=0.38, loss=2.44, step=6000]
Valid: 100% 6944/6944 [00:00<00:00, 15013.14 uttr/s, accuracy=0.35, loss=2.96]
Train: 100% 2000/2000 [00:04<00:00, 441.53 step/s, accuracy=0.56, loss=2.18, step=8000]
Valid: 100% 6944/6944 [00:00<00:00, 15975.02 uttr/s, accuracy=0.38, loss=2.76]
Train: 100% 2000/2000 [00:04<00:00, 453.62 step/s, accuracy=0.41, loss=2.79, step=1e+4]
Valid: 100% 6944/6944 [00:00<00:00, 16010.53 uttr/s, accuracy=0.42, loss=2.58]
Train:   4% 80/2000 [00:00<00:04, 400.07 step/s, accuracy=0.47, loss=2.13, step=10080]

Step 10000, best model saved. (accuracy=0.4243)


Train: 100% 2000/2000 [00:04<00:00, 436.18 step/s, accuracy=0.47, loss=2.50, step=12000]
Valid: 100% 6944/6944 [00:00<00:00, 15494.29 uttr/s, accuracy=0.44, loss=2.51]
Train: 100% 2000/2000 [00:04<00:00, 444.80 step/s, accuracy=0.56, loss=2.06, step=14000]
Valid: 100% 6944/6944 [00:00<00:00, 17118.04 uttr/s, accuracy=0.48, loss=2.28]
Train: 100% 2000/2000 [00:04<00:00, 442.09 step/s, accuracy=0.44, loss=2.74, step=16000]
Valid: 100% 6944/6944 [00:00<00:00, 18551.21 uttr/s, accuracy=0.49, loss=2.22]
Train: 100% 2000/2000 [00:04<00:00, 444.56 step/s, accuracy=0.53, loss=1.91, step=18000]
Valid: 100% 6944/6944 [00:00<00:00, 15794.70 uttr/s, accuracy=0.51, loss=2.15]
Train: 100% 2000/2000 [00:04<00:00, 449.19 step/s, accuracy=0.53, loss=2.04, step=2e+4] 
Valid: 100% 6944/6944 [00:00<00:00, 14520.71 uttr/s, accuracy=0.51, loss=2.14]
Train:   4% 81/2000 [00:00<00:04, 400.97 step/s, accuracy=0.78, loss=1.23, step=20080]

Step 20000, best model saved. (accuracy=0.5118)


...

Step 50000, best model saved. (accuracy=0.6470)


Train: 100% 2000/2000 [00:04<00:00, 453.27 step/s, accuracy=0.69, loss=1.44, step=52000]
Valid: 100% 6944/6944 [00:00<00:00, 16083.23 uttr/s, accuracy=0.66, loss=1.48]
Train: 100% 2000/2000 [00:04<00:00, 452.69 step/s, accuracy=0.59, loss=1.61, step=54000]
Valid: 100% 6944/6944 [00:00<00:00, 15368.22 uttr/s, accuracy=0.66, loss=1.43]
Train: 100% 2000/2000 [00:04<00:00, 440.25 step/s, accuracy=0.75, loss=1.02, step=56000]
Valid: 100% 6944/6944 [00:00<00:00, 16030.28 uttr/s, accuracy=0.66, loss=1.48]
Train: 100% 2000/2000 [00:04<00:00, 439.50 step/s, accuracy=0.66, loss=1.35, step=58000]
Valid: 100% 6944/6944 [00:00<00:00, 16201.89 uttr/s, accuracy=0.67, loss=1.44]
Train: 100% 2000/2000 [00:04<00:00, 437.01 step/s, accuracy=0.66, loss=1.10, step=6e+4] 
Valid: 100% 6944/6944 [00:00<00:00, 15095.31 uttr/s, accuracy=0.67, loss=1.42]
Train:   4% 86/2000 [00:00<00:04, 448.92 step/s, accuracy=0.78, loss=0.89, step=60086]

Step 60000, best model saved. (accuracy=0.6718)


Train: 100% 2000/2000 [00:04<00:00, 448.35 step/s, accuracy=0.72, loss=0.93, step=62000]
Valid: 100% 6944/6944 [00:00<00:00, 14548.32 uttr/s, accuracy=0.67, loss=1.43]
Train: 100% 2000/2000 [00:04<00:00, 441.33 step/s, accuracy=0.56, loss=2.00, step=64000]
Valid: 100% 6944/6944 [00:00<00:00, 14917.57 uttr/s, accuracy=0.67, loss=1.39]
Train: 100% 2000/2000 [00:04<00:00, 443.07 step/s, accuracy=0.78, loss=0.96, step=66000]
Valid: 100% 6944/6944 [00:00<00:00, 15263.90 uttr/s, accuracy=0.68, loss=1.42]
Train: 100% 2000/2000 [00:04<00:00, 440.73 step/s, accuracy=0.88, loss=0.44, step=68000]
Valid: 100% 6944/6944 [00:00<00:00, 15421.92 uttr/s, accuracy=0.67, loss=1.41]
Train: 100% 2000/2000 [00:04<00:00, 440.42 step/s, accuracy=0.75, loss=0.91, step=7e+4] 
Valid: 100% 6944/6944 [00:00<00:00, 17494.17 uttr/s, accuracy=0.68, loss=1.40]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 70000, best model saved. (accuracy=0.6766)


In [15]:

import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)


In [19]:
import json
import csv
from pathlib import Path
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "../../Data/ml2021spring-hw4/Dataset",
    "model_path": "./model.ckpt",
    "output_path": "./output.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spk=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])
  
  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!








































































100%|██████████| 6000/6000 [00:03<00:00, 1716.13it/s]
